<a href="https://colab.research.google.com/github/perezbrotonsluis/movie-tv-recommender/blob/main/notebooks/01_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objetivo del análisis
Analizar la estructura de los datasets y evaluar su idoneidad para un sistema recomendador.


## Observaciones iniciales
- Los géneros vienen como listas separadas por comas
- Existen valores nulos en la columna description
- Las tres plataformas comparten la mayoría de columnas

# Carga de datasets y primer estudio


Todo este proceso ha sido hecho con la ayuda de gemini.

Prompt:
Quiero importar los csv que tengo guardado en GitHub. Cómo puedo hacerlo?

Salida:

Entendido. Para manejar múltiples archivos CSV de GitHub de manera similar a cómo os.walk explora un directorio local, necesitaremos interactuar con la API de GitHub para listar los archivos. Luego, podemos construir las URLs 'raw' y usar pandas para importarlos. Aquí te propongo un plan:

- Configurar Acceso a GitHub: Obtener la información necesaria del repositorio (nombre de usuario/organización, nombre del repositorio, rama y ruta base donde se encuentran los archivos CSV) y preparar la forma de autenticarse o hacer peticiones a la API de GitHub, si es necesario.

- Listar Archivos CSV del Repositorio: Utilizar una librería Python (como requests) para interactuar con la API de GitHub y listar recursivamente todos los archivos CSV dentro de la ruta base especificada en el repositorio. Esto simulará la función de os.walk para encontrar los archivos.
- Construir URLs Raw para Cada CSV: Para cada archivo CSV identificado en el paso anterior, construir su URL 'raw' completa para que pandas pueda acceder directamente a su contenido.

- Importar Múltiples CSVs: Iterar sobre la lista de URLs 'raw' y usar pandas.read_csv() para importar cada archivo CSV a un DataFrame. Los DataFrames importados se pueden almacenar en una lista o un diccionario para su posterior procesamiento.

- Analizar la Estructura de los DataFrames Importados: Una vez importados todos los DataFrames, realizar un análisis inicial de su estructura, como verificar columnas, tipos de datos, valores nulos y cualquier otra observación relevante para un sistema recomendador, como se mencionó en el objetivo original.
Final Task: Resumir el proceso de importación y el análisis inicial de la estructura de los datasets de GitHub.

Generando...

Preparing

Configurar Acceso a GitHub

Listar Archivos CSV del Repositorio

Construir URLs Raw para Cada CSV

Importar Múltiples CSVs

Analizar la Estructura de los  DataFrames Importados

Final Task

## Configurar Acceso a GitHub

In [2]:
github_username = 'perezbrotonsluis'  # @param {type: "string"}
github_repository = 'movie-tv-recommender'  # @param {type: "string"}
github_branch = 'main'  # @param {type: "string"}
github_base_path = 'data/raw'  # @param {type: "string"} \# e.g., 'data/' or 'datasets/'

print(f"Usuario/Organización de GitHub: {github_username}")
print(f"Repositorio de GitHub: {github_repository}")
print(f"Rama de GitHub: {github_branch}")
print(f"Ruta base en el repositorio: {github_base_path if github_base_path else 'Raíz del repositorio'}")

Usuario/Organización de GitHub: perezbrotonsluis
Repositorio de GitHub: movie-tv-recommender
Rama de GitHub: main
Ruta base en el repositorio: data/raw


In [3]:
github_raw_url_base = f"https://raw.githubusercontent.com/{github_username}/{github_repository}/{github_branch}/"
print(f"URL base de GitHub Raw: {github_raw_url_base}")

URL base de GitHub Raw: https://raw.githubusercontent.com/perezbrotonsluis/movie-tv-recommender/main/


In [4]:
full_csv_base_url = f"{github_raw_url_base}{github_base_path}/" if github_base_path else github_raw_url_base
print(f"URL completa base para los archivos CSV: {full_csv_base_url}")

URL completa base para los archivos CSV: https://raw.githubusercontent.com/perezbrotonsluis/movie-tv-recommender/main/data/raw/


In [5]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.5 MB/s eta 0:00:00


In [6]:
from github import Github
from google.colab import userdata

github_token = userdata.get('github-token') # Your GitHub Personal Access Token (PAT) here

# Initialize the GitHub API client
if github_token:
    g = Github(github_token)
    print("GitHub client initialized with token.")
else:
    g = Github()
    print("GitHub client initialized without token (unauthenticated). Rate limits will apply.")


GitHub client initialized with token.


/tmp/ipython-input-2757104653.py:8: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(github_token)


## Listar Archivos CSV del Repositorio

In [7]:
try:
    # Get the repository object
    repo = g.get_user().get_repo(github_repository) # Assumes the repo belongs to the authenticated user
    # If the repository is owned by an organization or another user:
    # repo = g.get_user(github_username).get_repo(github_repository)

    csv_files = []

    # Function to recursively list files
    def list_files_in_repo(repo, path, branch_name):
        contents = repo.get_contents(path, ref=branch_name)
        for content_file in contents:
            if content_file.type == 'dir':
                list_files_in_repo(repo, content_file.path, branch_name)
            else:
                if content_file.name.endswith('.csv'):
                    # Store the full path within the repository
                    csv_files.append(content_file.path)

    # Start listing from the specified base path
    list_files_in_repo(repo, github_base_path, github_branch)

    print(f"Archivos CSV encontrados en '{github_base_path}' en la rama '{github_branch}':")
    if csv_files:
        for file_path in csv_files:
            print(f"- {file_path}")
    else:
        print("No se encontraron archivos CSV.")

except Exception as e:
    print(f"Ocurrió un error al listar los archivos del repositorio: {e}")
    print("Asegúrate de que el nombre de usuario/organización, el nombre del repositorio y la rama sean correctos.")
    if github_token == '':
        print("Considera proporcionar un Token de Acceso Personal de GitHub para evitar problemas de límites de tasa.")

Archivos CSV encontrados en 'data/raw' en la rama 'main':
- data/raw/AmazonPrime_credits.csv
- data/raw/AmazonPrime_titles.csv
- data/raw/Disney_credits.csv
- data/raw/Disney_titles.csv
- data/raw/HBO_credits.csv
- data/raw/HBO_titles.csv
- data/raw/Netflix_credits.csv
- data/raw/Netflix_titles.csv


## Construir URLs Raw para Cada CSV

In [8]:
full_csv_raw_urls = []

for csv_file_path in csv_files:
    full_url = f"{github_raw_url_base}{csv_file_path}"
    full_csv_raw_urls.append(full_url)

print("URLs Raw completas para los archivos CSV:")
if full_csv_raw_urls:
    for url in full_csv_raw_urls:
        print(f"- {url}")
else:
    print("No se pudieron construir las URLs raw para los archivos CSV.")

URLs Raw completas para los archivos CSV:
- https://raw.githubusercontent.com/perezbrotonsluis/movie-tv-recommender/main/data/raw/AmazonPrime_credits.csv
- https://raw.githubusercontent.com/perezbrotonsluis/movie-tv-recommender/main/data/raw/AmazonPrime_titles.csv
- https://raw.githubusercontent.com/perezbrotonsluis/movie-tv-recommender/main/data/raw/Disney_credits.csv
- https://raw.githubusercontent.com/perezbrotonsluis/movie-tv-recommender/main/data/raw/Disney_titles.csv
- https://raw.githubusercontent.com/perezbrotonsluis/movie-tv-recommender/main/data/raw/HBO_credits.csv
- https://raw.githubusercontent.com/perezbrotonsluis/movie-tv-recommender/main/data/raw/HBO_titles.csv
- https://raw.githubusercontent.com/perezbrotonsluis/movie-tv-recommender/main/data/raw/Netflix_credits.csv
- https://raw.githubusercontent.com/perezbrotonsluis/movie-tv-recommender/main/data/raw/Netflix_titles.csv


## Importar Múltiples CSVs

### Subtask:
Iterar sobre la lista de URLs 'raw' y usar `pandas.read_csv()` para importar cada archivo CSV a un DataFrame, almacenándolos en un diccionario.


In [9]:
import pandas as pd
import os # Import os module for path manipulation

# Initialize an empty dictionary to store the DataFrames
dataframes = {}

# Iterate over each raw URL
for url in full_csv_raw_urls:
    try:
        # Extract the filename from the URL
        filename = os.path.basename(url)

        # Extract the platform name (e.g., 'AmazonPrime')
        # Assumes format like 'Platform_credits.csv' or 'Platform_titles.csv'
        platform_name = filename.split('_')[0]

        # Read the CSV into a DataFrame
        df = pd.read_csv(url)

        # Store the DataFrame in the dictionary with the platform name as key
        # If there are multiple files per platform (e.g., credits and titles),
        # we need to decide how to handle the key. For now, we'll append the filename
        # or differentiate. Let's make the key the full filename without extension for clarity.
        # E.g., 'AmazonPrime_credits' or 'AmazonPrime_titles'
        key = os.path.splitext(filename)[0]

        dataframes[key] = df
        print(f"Successfully imported '{filename}' as '{key}'")
    except Exception as e:
        print(f"Error importing {url}: {e}")

# Print the keys of the dataframes dictionary to verify
print("\nDataFrames successfully loaded with keys:")
for key in dataframes.keys():
    print(f"- {key}")

# Display the head of one of the DataFrames to confirm (e.g., the first one found)
if dataframes:
    first_key = list(dataframes.keys())[0]
    print(f"\nHead of the first loaded DataFrame ('{first_key}'):")
    display(dataframes[first_key].head())

Successfully imported 'AmazonPrime_credits.csv' as 'AmazonPrime_credits'
Successfully imported 'AmazonPrime_titles.csv' as 'AmazonPrime_titles'
Successfully imported 'Disney_credits.csv' as 'Disney_credits'
Successfully imported 'Disney_titles.csv' as 'Disney_titles'
Successfully imported 'HBO_credits.csv' as 'HBO_credits'
Successfully imported 'HBO_titles.csv' as 'HBO_titles'
Successfully imported 'Netflix_credits.csv' as 'Netflix_credits'
Successfully imported 'Netflix_titles.csv' as 'Netflix_titles'

DataFrames successfully loaded with keys:
- AmazonPrime_credits
- AmazonPrime_titles
- Disney_credits
- Disney_titles
- HBO_credits
- HBO_titles
- Netflix_credits
- Netflix_titles

Head of the first loaded DataFrame ('AmazonPrime_credits'):


,person_id,id,name,character,role
0,59401,ts20945,Joe Besser,Joe,ACTOR
1,31460,ts20945,Moe Howard,Moe,ACTOR
2,31461,ts20945,Larry Fine,Larry,ACTOR
3,21174,tm19248,Buster Keaton,Johnny Gray,ACTOR
4,28713,tm19248,Marion Mack,Annabelle Lee,ACTOR


## Analizar la Estructura de los DataFrames Importados

Este análisis no se hizo mediante gemini. fue prorpio. explicar bien con gemini

Explicar aqui brevemente que se realizará un recomendador basado en contenido. Por eso ...

In [ ]:
🧠 Idea clave del EDA
En esta fase no limpias, no imputas y no decides aún qué borrar.
Solo evalúas riesgos para el modelo.
Así que la pregunta correcta no es “¿hay nulos?”, sino:
¿Estos nulos rompen o debilitan el recomendador?
🎯 1. Columnas CRÍTICAS para tu recomendador
Estas son las que sí o sí importan, porque sin ellas el sistema no funciona o pierde sentido.
✅ id
Es la clave para unir títulos con créditos
No puede tener nulos
Si hay nulos → ese registro es inútil
✅ title
Necesaria para:
identificar el ítem
mostrar recomendaciones
Nulos aquí → no sabes qué estás recomendando
Es un criterio claro para eliminar filas
✅ description
Es la fuente principal de contexto semántico
Nulos aquí:
no rompen el sistema
pero empobrecen la similitud
Decisión habitual:
mantenerlos y dejar descripción vacía
o eliminarlos si son pocos
✔️ Importan mucho
❌ Pero no obligan a eliminar automáticamente
✅ show_type (película / serie)
No entra en TF-IDF
Pero sirve para:
análisis
filtros
coherencia de recomendaciones
Nulos aquí:
no rompen el modelo
pero conviene revisarlos
🟡 2. Columnas IMPORTANTES pero NO críticas
Estas mejoran, pero no son imprescindibles.
⚠️ genres
Aporta mucha información
Pero:
a veces está mal formateada
puede estar vacía
Si hay nulos:
el modelo sigue funcionando
simplemente usa menos señales
👉 No elimines por esto
⚠️ credits (actores / director)
En credits.csv:
id → crítico
name → importante
role → útil para filtrar, no para el modelo
Nulos en:
character_name → irrelevantes
algunos name → revisables, pero no dramáticos
🟢 3. Columnas POCO relevantes para el modelo
Estas no te deberían preocupar ahora.
runtime
seasons
production_countries
imdb_score
tmdb_score
imdb_votes
tmdb_popularity
age_certification
Nulos aquí:
no afectan al TF-IDF
no justifican eliminación
se pueden mencionar como limitación del dataset
📋 4. Qué deberías anotar en el EDA (esto es oro para la memoria)
En el notebook, algo así:
### Análisis de valores nulos
- No se detectan valores nulos en la columna `id`
- Existen valores nulos en `description` (~5%), lo que puede reducir el contexto semántico en algunos títulos
- La columna `genres` presenta valores faltantes en un porcentaje reducido
- Los valores nulos en columnas numéricas no afectan al modelo de recomendación

### Funciones para el análisis

In [23]:
def analisis_df(df):
  # Mostrar las primeras 5 filas
  print(f"\nHead():")
  display(df.head())

  # Mostrar número de filas y columnas
  print(f"\nShape:")
  display(df.shape)

  # Mostrar un resumen del DataFrame
  print(f"\nInfo():")
  df.info()

  # Generar estadísticas descriptivas para columnas numéricas (.describe())
  print(f"\nDescribe():")
  display(df.describe())

  # Mostrar la cantidad de valores duplicados
  print(f"\nDuplicated():")
  display(df.duplicated().sum())

  # Mostrar la cantidad de valores nulos por columna
  print(f"\nIsnull().sum():")
  display(df.isnull().sum())

### Dataset Netflix

In [24]:
analisis_df(dataframes['Netflix_titles'])


Head():


,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600



Shape:


(5850, 15)


Info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,5850.000000,5850.000000,2106.000000,5368.000000,5.352000e+03,5759.000000,5539.000000
mean,2016.417094,76.888889,2.162868,6.510861,2.343938e+04,22.637925,6.829175
std,6.937726,39.002509,2.689041,1.163826,9.582047e+04,81.680263,1.170391
min,1945.000000,0.000000,1.000000,1.500000,5.000000e+00,0.009442,0.500000
25%,2016.000000,44.000000,1.000000,5.800000,5.167500e+02,2.728500,6.100000
50%,2018.000000,83.000000,1.000000,6.600000,2.233500e+03,6.821000,6.900000
75%,2020.000000,104.000000,2.000000,7.300000,9.494000e+03,16.590000,7.537500
max,2022.000000,240.000000,42.000000,9.600000,2.294231e+06,2274.044000,10.000000



Duplicated():


np.int64(0)


Isnull().sum():


,0
id,0
title,1
type,0
description,18
release_year,0
age_certification,2619
runtime,0
genres,0
production_countries,0
seasons,3744


revisar columnas mas importantes. si no hay nulos genial. si hay nulos valorar que e sun porblema. si suponen un porcentaje muy pequeño, se podrian borrar en el futuro. Así con todo.

In [25]:
analisis_df(dataframes['Netflix_credits'])


Head():


,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR



Shape:


(77801, 5)


Info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB

Describe():


,person_id
count,7.780100e+04
mean,5.294888e+05
std,6.430167e+05
min,7.000000e+00
25%,4.530600e+04
50%,1.983580e+05
75%,8.880960e+05
max,2.462818e+06



Duplicated():


np.int64(0)


Isnull().sum():


,0
person_id,0
id,0
name,0
character,9772
role,0


HACER ANALISIS INDIVIDUAL

### Dataset HBO

In [26]:
analisis_df(dataframes['HBO_titles'])


Head():


,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,tm155702,The Wizard of Oz,MOVIE,Young Dorothy finds herself in a magical world...,1939,G,102,"['fantasy', 'family']",['US'],NaN,tt0032138,8.1,389774.0,41.442,7.6
1,tm83648,Citizen Kane,MOVIE,"Newspaper magnate, Charles Foster Kane is take...",1941,PG,119,['drama'],['US'],NaN,tt0033467,8.3,433804.0,14.383,8.0
2,tm77588,Casablanca,MOVIE,"In Casablanca, Morocco in December 1941, a cyn...",1942,PG,102,"['drama', 'romance', 'war']",['US'],NaN,tt0034583,8.5,558849.0,20.087,8.2
3,tm82363,The Big Sleep,MOVIE,Private Investigator Philip Marlowe is hired b...,1946,NaN,116,"['thriller', 'crime']",['US'],NaN,tt0038355,7.9,84494.0,12.911,7.7
4,tm84701,The Maltese Falcon,MOVIE,A private detective takes on a case that invol...,1941,NaN,100,"['thriller', 'romance', 'crime']",['US'],NaN,tt0033870,8.0,156603.0,12.788,7.8



Shape:


(3294, 15)


Info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3294 entries, 0 to 3293
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3294 non-null   object 
 1   title                 3294 non-null   object 
 2   type                  3294 non-null   object 
 3   description           3282 non-null   object 
 4   release_year          3294 non-null   int64  
 5   age_certification     2086 non-null   object 
 6   runtime               3294 non-null   int64  
 7   genres                3294 non-null   object 
 8   production_countries  3294 non-null   object 
 9   seasons               756 non-null    float64
 10  imdb_id               2968 non-null   object 
 11  imdb_score            2922 non-null   float64
 12  imdb_votes            2911 non-null   float64
 13  tmdb_popularity       3261 non-null   float64
 14  tmdb_score            3026 non-null   float64
dtypes: float64(5

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,3294.000000,3294.000000,756.000000,2922.000000,2.911000e+03,3261.000000,3026.000000
mean,2001.194293,80.914997,2.998677,6.883949,6.461835e+04,21.105891,6.850099
std,23.542337,38.475347,3.651181,1.059668,1.754617e+05,89.137800,1.081377
min,1901.000000,1.000000,1.000000,1.500000,5.000000e+00,0.050393,1.000000
25%,1995.000000,51.250000,1.000000,6.300000,9.245000e+02,3.450000,6.200000
50%,2010.000000,89.000000,2.000000,7.000000,8.071000e+03,8.561000,6.900000
75%,2018.000000,106.000000,4.000000,7.600000,4.764500e+04,17.711000,7.500000
max,2022.000000,255.000000,52.000000,9.400000,2.555504e+06,4447.894000,10.000000



Duplicated():


np.int64(0)


Isnull().sum():


,0
id,0
title,0
type,0
description,12
release_year,0
age_certification,1208
runtime,0
genres,0
production_countries,0
seasons,2538


In [27]:
analisis_df(dataframes['HBO_credits'])


Head():


,person_id,id,name,character,role
0,60017,tm155702,Judy Garland,Dorothy Gale,ACTOR
1,53496,tm155702,Ray Bolger,Hunk / Scarecrow,ACTOR
2,79549,tm155702,Jack Haley,Hickory / Tin Man,ACTOR
3,79548,tm155702,Bert Lahr,Zeke / Cowardly Lion,ACTOR
4,60995,tm155702,Margaret Hamilton,Elmira Gulch / Wicked Witch of the West,ACTOR



Shape:


(66393, 5)


Info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66393 entries, 0 to 66392
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  66393 non-null  int64 
 1   id         66393 non-null  object
 2   name       66393 non-null  object
 3   character  61888 non-null  object
 4   role       66393 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.5+ MB

Describe():


,person_id
count,6.639300e+04
mean,3.479124e+05
std,5.539732e+05
min,7.000000e+00
25%,1.754400e+04
50%,7.037900e+04
75%,4.835320e+05
max,2.369354e+06



Duplicated():


np.int64(0)


Isnull().sum():


,0
person_id,0
id,0
name,0
character,4505
role,0


### Dataset Amazon Prime

In [28]:
analisis_df(dataframes['AmazonPrime_titles'])


Head():


,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts20945,The Three Stooges,SHOW,The Three Stooges were an American vaudeville ...,1934,TV-PG,19,"['comedy', 'family', 'animation', 'action', 'f...",['US'],26.0,tt0850645,8.6,1092.0,15.424,7.6
1,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,78,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],NaN,tt0017925,8.2,89766.0,8.647,8.0
2,tm82253,The Best Years of Our Lives,MOVIE,It's the hope that sustains the spirit of ever...,1946,NaN,171,"['romance', 'war', 'drama']",['US'],NaN,tt0036868,8.1,63026.0,8.435,7.8
3,tm83884,His Girl Friday,MOVIE,"Hildy, the journalist former wife of newspaper...",1940,NaN,92,"['comedy', 'drama', 'romance']",['US'],NaN,tt0032599,7.8,57835.0,11.270,7.4
4,tm56584,In a Lonely Place,MOVIE,An aspiring actress begins to suspect that her...,1950,NaN,94,"['thriller', 'drama', 'romance']",['US'],NaN,tt0042593,7.9,30924.0,8.273,7.6



Shape:


(9871, 15)


Info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9871 entries, 0 to 9870
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    9871 non-null   object 
 1   title                 9871 non-null   object 
 2   type                  9871 non-null   object 
 3   description           9752 non-null   object 
 4   release_year          9871 non-null   int64  
 5   age_certification     3384 non-null   object 
 6   runtime               9871 non-null   int64  
 7   genres                9871 non-null   object 
 8   production_countries  9871 non-null   object 
 9   seasons               1357 non-null   float64
 10  imdb_id               9204 non-null   object 
 11  imdb_score            8850 non-null   float64
 12  imdb_votes            8840 non-null   float64
 13  tmdb_popularity       9324 non-null   float64
 14  tmdb_score            7789 non-null   float64
dtypes: float64(5

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,9871.000000,9871.000000,1357.000000,8850.000000,8.840000e+03,9324.000000,7789.000000
mean,2001.327221,85.973052,2.791452,5.976395,8.533614e+03,6.910204,5.984247
std,25.810071,33.512466,4.148958,1.343842,4.592015e+04,30.004098,1.517986
min,1912.000000,1.000000,1.000000,1.100000,5.000000e+00,0.000011,0.800000
25%,1995.500000,65.000000,1.000000,5.100000,1.170000e+02,1.232000,5.100000
50%,2014.000000,89.000000,1.000000,6.100000,4.625000e+02,2.536000,6.000000
75%,2018.000000,102.000000,3.000000,6.900000,2.236250e+03,5.634000,6.900000
max,2022.000000,549.000000,51.000000,9.900000,1.133692e+06,1437.906000,10.000000



Duplicated():


np.int64(3)


Isnull().sum():


,0
id,0
title,0
type,0
description,119
release_year,0
age_certification,6487
runtime,0
genres,0
production_countries,0
seasons,8514


In [29]:
analisis_df(dataframes['AmazonPrime_credits'])


Head():


,person_id,id,name,character,role
0,59401,ts20945,Joe Besser,Joe,ACTOR
1,31460,ts20945,Moe Howard,Moe,ACTOR
2,31461,ts20945,Larry Fine,Larry,ACTOR
3,21174,tm19248,Buster Keaton,Johnny Gray,ACTOR
4,28713,tm19248,Marion Mack,Annabelle Lee,ACTOR



Shape:


(124235, 5)


Info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124235 entries, 0 to 124234
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   person_id  124235 non-null  int64 
 1   id         124235 non-null  object
 2   name       124235 non-null  object
 3   character  107948 non-null  object
 4   role       124235 non-null  object
dtypes: int64(1), object(4)
memory usage: 4.7+ MB

Describe():


,person_id
count,1.242350e+05
mean,4.064737e+05
std,5.616296e+05
min,1.000000e+00
25%,3.899200e+04
50%,1.339490e+05
75%,5.712560e+05
max,2.371153e+06



Duplicated():


np.int64(56)


Isnull().sum():


,0
person_id,0
id,0
name,0
character,16287
role,0


### Dataset Disney

In [30]:
analisis_df(dataframes['Disney_titles'])


Head():


,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,tm74391,Fantasia,MOVIE,Walt Disney's timeless masterpiece is an extra...,1940,G,120,"['animation', 'family', 'music', 'fantasy']",['US'],NaN,tt0032455,7.7,94681.0,57.751,7.4
1,tm67803,Snow White and the Seven Dwarfs,MOVIE,"A beautiful girl, Snow White, takes refuge in ...",1937,G,83,"['fantasy', 'family', 'romance', 'animation', ...",['US'],NaN,tt0029583,7.6,195321.0,107.137,7.1
2,tm82546,Pinocchio,MOVIE,Lonely toymaker Geppetto has his wishes answer...,1940,G,88,"['animation', 'comedy', 'family', 'fantasy']",['US'],NaN,tt0032910,7.5,141937.0,71.160,7.1
3,tm79357,Bambi,MOVIE,Bambi's tale unfolds from season to season as ...,1942,G,70,"['animation', 'drama', 'family']",['US'],NaN,tt0034492,7.3,140406.0,68.136,7.0
4,tm62671,Treasure Island,MOVIE,Enchanted by the idea of locating treasure bur...,1950,PG,96,"['family', 'action']","['GB', 'US']",NaN,tt0043067,6.9,8229.0,10.698,6.5



Shape:


(1535, 15)


Info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1535 entries, 0 to 1534
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1535 non-null   object 
 1   title                 1535 non-null   object 
 2   type                  1535 non-null   object 
 3   description           1529 non-null   object 
 4   release_year          1535 non-null   int64  
 5   age_certification     1210 non-null   object 
 6   runtime               1535 non-null   int64  
 7   genres                1535 non-null   object 
 8   production_countries  1535 non-null   object 
 9   seasons               415 non-null    float64
 10  imdb_id               1133 non-null   object 
 11  imdb_score            1108 non-null   float64
 12  imdb_votes            1105 non-null   float64
 13  tmdb_popularity       1524 non-null   float64
 14  tmdb_score            1426 non-null   float64
dtypes: float64(5

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,1535.000000,1535.000000,415.000000,1108.000000,1.105000e+03,1524.000000,1426.000000
mean,2003.577850,60.163518,2.583133,6.605144,7.635004e+04,40.238486,6.916339
std,21.678528,39.904824,3.013208,1.068985,1.885893e+05,263.149723,1.022268
min,1928.000000,1.000000,1.000000,1.600000,5.000000e+00,0.600000,1.000000
25%,1999.000000,23.000000,1.000000,5.900000,4.900000e+02,3.870750,6.300000
50%,2012.000000,53.000000,2.000000,6.600000,4.219000e+03,9.383500,6.900000
75%,2019.000000,93.000000,3.000000,7.400000,4.163900e+04,25.436750,7.500000
max,2022.000000,182.000000,34.000000,9.700000,1.353907e+06,9323.832000,10.000000



Duplicated():


np.int64(0)


Isnull().sum():


,0
id,0
title,0
type,0
description,6
release_year,0
age_certification,325
runtime,0
genres,0
production_countries,0
seasons,1120


In [31]:
analisis_df(dataframes['Disney_credits'])


Head():


,person_id,id,name,character,role
0,23433,tm74391,Deems Taylor,Narrator - Narrative Introductions,ACTOR
1,5910,tm74391,Walt Disney,Mickey Mouse (segment 'The Sorcerer's Apprenti...,ACTOR
2,23436,tm74391,Julietta Novis,Soloist (segment 'Ave Maria') (singing voice),ACTOR
3,23434,tm74391,Leopold Stokowski,Himself - Conductor of The Philadelphia Orchestra,ACTOR
4,23441,tm74391,Paul Satterfield,NaN,DIRECTOR



Shape:


(26412, 5)


Info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26412 entries, 0 to 26411
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  26412 non-null  int64 
 1   id         26412 non-null  object
 2   name       26412 non-null  object
 3   character  24769 non-null  object
 4   role       26412 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.0+ MB

Describe():


,person_id
count,2.641200e+04
mean,3.233141e+05
std,5.266726e+05
min,4.800000e+01
25%,1.160575e+04
50%,4.663300e+04
75%,4.417018e+05
max,2.364007e+06



Duplicated():


np.int64(0)


Isnull().sum():


,0
person_id,0
id,0
name,0
character,1643
role,0


## Unificación de DataFrames de Títulos y Créditos

Primero, separaremos los DataFrames en dos grupos: aquellos que contienen información de títulos y aquellos que contienen información de créditos. Luego, concatenaremos cada grupo para formar dos DataFrames maestros: `all_titles_df` y `all_credits_df`.

In [13]:
# Listas para almacenar los DataFrames de títulos y créditos por separado
titles_dfs = []
credits_dfs = []

# Iterar sobre el diccionario de DataFrames y clasificarlos
for key, df in dataframes.items():
    if '_titles' in key:
        # Añadir una columna 'platform' para identificar la plataforma de origen
        df['platform'] = key.replace('_titles', '')
        titles_dfs.append(df)
    elif '_credits' in key:
        # Añadir una columna 'platform' para identificar la plataforma de origen
        df['platform'] = key.replace('_credits', '')
        credits_dfs.append(df)

# Concatenar todos los DataFrames de títulos
all_titles_df = pd.concat(titles_dfs, ignore_index=True)
print(f"DataFrame consolidado de títulos creado con {len(all_titles_df)} filas.")

# Concatenar todos los DataFrames de créditos
all_credits_df = pd.concat(credits_dfs, ignore_index=True)
print(f"DataFrame consolidado de créditos creado con {len(all_credits_df)} filas.")

DataFrame consolidado de títulos creado con 20550 filas.
DataFrame consolidado de créditos creado con 294841 filas.


### Análisis del DataFrame Consolidado de Títulos (`all_titles_df`)

Ahora, realizaremos un análisis similar al inicial para el nuevo DataFrame consolidado de títulos.

In [14]:
print("\n--- Análisis para DataFrame: all_titles_df ---")

print("\nall_titles_df.head():")
display(all_titles_df.head())

print("\nall_titles_df.info():")
all_titles_df.info()

print("\nall_titles_df.describe():")
display(all_titles_df.describe())

print("\nall_titles_df.isnull().sum():")
display(all_titles_df.isnull().sum()[all_titles_df.isnull().sum() > 0]) # Display only columns with nulls


--- Análisis para DataFrame: all_titles_df ---

all_titles_df.head():


,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,platform
0,ts20945,The Three Stooges,SHOW,The Three Stooges were an American vaudeville ...,1934,TV-PG,19,"['comedy', 'family', 'animation', 'action', 'f...",['US'],26.0,tt0850645,8.6,1092.0,15.424,7.6,AmazonPrime
1,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,78,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],NaN,tt0017925,8.2,89766.0,8.647,8.0,AmazonPrime
2,tm82253,The Best Years of Our Lives,MOVIE,It's the hope that sustains the spirit of ever...,1946,NaN,171,"['romance', 'war', 'drama']",['US'],NaN,tt0036868,8.1,63026.0,8.435,7.8,AmazonPrime
3,tm83884,His Girl Friday,MOVIE,"Hildy, the journalist former wife of newspaper...",1940,NaN,92,"['comedy', 'drama', 'romance']",['US'],NaN,tt0032599,7.8,57835.0,11.270,7.4,AmazonPrime
4,tm56584,In a Lonely Place,MOVIE,An aspiring actress begins to suspect that her...,1950,NaN,94,"['thriller', 'drama', 'romance']",['US'],NaN,tt0042593,7.9,30924.0,8.273,7.6,AmazonPrime



all_titles_df.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20550 entries, 0 to 20549
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    20550 non-null  object 
 1   title                 20549 non-null  object 
 2   type                  20550 non-null  object 
 3   description           20395 non-null  object 
 4   release_year          20550 non-null  int64  
 5   age_certification     9911 non-null   object 
 6   runtime               20550 non-null  int64  
 7   genres                20550 non-null  object 
 8   production_countries  20550 non-null  object 
 9   seasons               4634 non-null   float64
 10  imdb_id               18752 non-null  object 
 11  imdb_score            18248 non-null  float64
 12  imdb_votes            18208 non-null  float64
 13  tmdb_popularity       19868 non-null  float64
 14  tmdb_score            17780 non-null  float64
 

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,20550.000000,20550.000000,4634.000000,18248.000000,1.820800e+04,19868.000000,17780.000000
mean,2005.769684,80.648418,2.520932,6.317120,2.599712e+04,16.355563,6.469584
std,22.428473,37.115032,3.380039,1.283395,1.064973e+05,95.232215,1.379463
min,1901.000000,0.000000,1.000000,1.100000,5.000000e+00,0.000011,0.500000
25%,2004.000000,53.000000,1.000000,5.500000,2.150000e+02,1.798000,5.800000
50%,2016.000000,87.000000,1.000000,6.400000,1.200000e+03,4.334500,6.600000
75%,2019.000000,103.000000,3.000000,7.200000,7.640750e+03,11.273250,7.300000
max,2022.000000,549.000000,52.000000,9.900000,2.555504e+06,9323.832000,10.000000



all_titles_df.isnull().sum():


,0
title,1
description,155
age_certification,10639
seasons,15916
imdb_id,1798
imdb_score,2302
imdb_votes,2342
tmdb_popularity,682
tmdb_score,2770


### Análisis del DataFrame Consolidado de Créditos (`all_credits_df`)

Finalmente, analizaremos el DataFrame consolidado de créditos.

In [15]:
print("\n--- Análisis para DataFrame: all_credits_df ---")

print("\nall_credits_df.head():")
display(all_credits_df.head())

print("\nall_credits_df.info():")
all_credits_df.info()

print("\nall_credits_df.describe():")
display(all_credits_df.describe())

print("\nall_credits_df.isnull().sum():")
display(all_credits_df.isnull().sum()[all_credits_df.isnull().sum() > 0]) # Display only columns with nulls


--- Análisis para DataFrame: all_credits_df ---

all_credits_df.head():


,person_id,id,name,character,role,platform
0,59401,ts20945,Joe Besser,Joe,ACTOR,AmazonPrime
1,31460,ts20945,Moe Howard,Moe,ACTOR,AmazonPrime
2,31461,ts20945,Larry Fine,Larry,ACTOR,AmazonPrime
3,21174,tm19248,Buster Keaton,Johnny Gray,ACTOR,AmazonPrime
4,28713,tm19248,Marion Mack,Annabelle Lee,ACTOR,AmazonPrime



all_credits_df.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294841 entries, 0 to 294840
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   person_id  294841 non-null  int64 
 1   id         294841 non-null  object
 2   name       294841 non-null  object
 3   character  262634 non-null  object
 4   role       294841 non-null  object
 5   platform   294841 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.5+ MB

all_credits_df.describe():


,person_id
count,2.948410e+05
mean,4.182978e+05
std,5.841311e+05
min,1.000000e+00
25%,2.820600e+04
50%,1.136800e+05
75%,6.122360e+05
max,2.462818e+06



all_credits_df.isnull().sum():


,0
character,32207


In [ ]:
### Esto va a ser un notbook solo de unificacion de dataset y de analisis muy breve de si todas las columnas son iguales par apoder unificarlos todos.
# Después ya unificarlos y hacer estudio. Comenzar mirando si hay duplicados por titulo y si es así eliminar uno de ellos. y luego ya estduio normal de taamñaos, nulos, formatos, etc